<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#ImageDataGenerator" data-toc-modified-id="ImageDataGenerator-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>ImageDataGenerator</a></span></li></ul></li><li><span><a href="#Preprocessing-images" data-toc-modified-id="Preprocessing-images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing images</a></span><ul class="toc-item"><li><span><a href="#Load-image-data-from-the-hierarchical-file-'chest_xray'-using-an-image-datagenerator" data-toc-modified-id="Load-image-data-from-the-hierarchical-file-'chest_xray'-using-an-image-datagenerator-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load image data from the hierarchical file 'chest_xray' using an image datagenerator</a></span><ul class="toc-item"><li><span><a href="#Define-the-X,y-of-Dataset" data-toc-modified-id="Define-the-X,y-of-Dataset-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Define the X,y of Dataset</a></span></li><li><span><a href="#Build-Base-Model" data-toc-modified-id="Build-Base-Model-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Build Base Model</a></span><ul class="toc-item"><li><span><a href="#Designing-the-CNN" data-toc-modified-id="Designing-the-CNN-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Designing the CNN</a></span></li></ul></li><li><span><a href="#Pooling-Layer:" data-toc-modified-id="Pooling-Layer:-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Pooling Layer:</a></span></li><li><span><a href="#Training-and-Evaluating-the-Model" data-toc-modified-id="Training-and-Evaluating-the-Model-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Training and Evaluating the Model</a></span></li></ul></li><li><span><a href="#Severe-OverFitting" data-toc-modified-id="Severe-OverFitting-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Severe OverFitting</a></span></li></ul></li></ul></div>

In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from random import sample
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models,layers
from tensorflow.keras.layers import Input, Lambda, Flatten,Activation, Dense, BatchNormalization, Conv2D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import optimizers



### ImageDataGenerator
https://keras.io/api/preprocessing/image/



## Preprocessing images


### Load image data from the hierarchical file 'chest_xray' using an image datagenerator
* Load images
* Read the picture files
* define testing,training , validation.
* Decode the JPEG content to RGB grids of pixels.
* reshape all images to same size of  28 x 28 pixels 
* Convert pixels into floating point tensors
* chuck data by "batch_size=32'
* Rescale the pixel values( between o and 255) to the [0,1] interval.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
original_start = datetime.datetime.now()
start = datetime.datetime.now()

In [3]:
train_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/data/train')
test_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/data/test')
val_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/data/val')

In [4]:
# Get all the data in the directory data/test , and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')
# Get all the data in the directory data/train, and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')
# Get all the data in the directory data/val, and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')




Found 624 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [6]:
cnn = Sequential()
#Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))
#Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))
# 2nd Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu"))
# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))
# Flatten the layer
cnn.add(Flatten())
# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 512))
cnn.add(Dense(activation = 'sigmoid', units = 1))


In [8]:
# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3211776   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5

In [9]:
cnn_history = cnn.fit_generator(train_generator,
                         steps_per_epoch = 100,
                         epochs = 25,
                         validation_data = val_generator,
                         validation_steps = 550)

Epoch 1/25
100/100 [==============================] - 165s 2s/step - loss: 0.3201 - accuracy: 0.8603 - val_loss: 0.3834 - val_accuracy: 0.8750
Epoch 2/25
100/100 [==============================] - 169s 2s/step - loss: 0.1381 - accuracy: 0.9472 - val_loss: 0.2579 - val_accuracy: 0.9375
Epoch 3/25
100/100 [==============================] - 161s 2s/step - loss: 0.1137 - accuracy: 0.9603 - val_loss: 0.4141 - val_accuracy: 0.7500
Epoch 4/25
100/100 [==============================] - 158s 2s/step - loss: 0.1007 - accuracy: 0.9663 - val_loss: 0.1843 - val_accuracy: 0.9375
Epoch 5/25
100/100 [==============================] - 157s 2s/step - loss: 0.0824 - accuracy: 0.9700 - val_loss: 0.1579 - val_accuracy: 1.0000
Epoch 6/25
100/100 [==============================] - 157s 2s/step - loss: 0.0784 - accuracy: 0.9734 - val_loss: 0.2707 - val_accuracy: 0.8750
Epoch 7/25
100/100 [==============================] - 163s 2s/step - loss: 0.0557 - accuracy: 0.9791 - val_loss: 0.4998 - val_accuracy: 0.8125

In [3]:
#### Load images
train_dir =('/Users/khuloodnasher/Documents/Flatiorn/imageclassification/image-classification-project/chest_xray/train')
test_dir =('/Users/khuloodnasher/Documents/Flatiorn/imageclassification/image-classification-project/chest_xray/test')
validation_dir =('/Users/khuloodnasher/Documents/Flatiorn/imageclassification/image-classification-project/chest_xray/val')
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(64, 64),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


#### Define the X,y of Dataset

In [4]:
# create the data sets
#train_images, train_labels = next(train_generator)
#test_images, test_labels = next(test_generator)
#val_images, val_labels = next(val_generator)

#### Build Base Model 
##### Designing the CNN
###### To design CNN using Keras,we must consider the following:

* alternate convolutional and pooling layers.

* have later layers having a larger number of parameters in order to detect more abstract patterns.

* Add some final dense layers to add a classifier to the convolutional base.

* Compile this model


#### Pooling Layer:
Pooling layers goal is to subsample (i.e., shrink) the input image in order to reduce the computational load, the memory usage, and the number of parameters (thereby limiting the risk of overfitting)
1. Max pooling: The maximum pixel value of the batch is selected.
2. Average pooling: The average value of all the pixels in the batch is selected.

In [5]:
# Always start the model with sequebtial function
model = models.Sequential()

# start adding layers and define the activation function of each layer as' relu', we define images in size 150x150
# in keras function for the convolution step is Conv2D

# input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))

#  First Hidden 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Second Hidden Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#  Third Hidden Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#Change all the layers dimension to vector through Fltten function
model.add(layers.Flatten())
# Adding the last layer before the output layer
model.add(layers.Dense(512, activation='relu'))
# Adding the output layer
model.add(layers.Dense(1, activation='sigmoid'))

# compiling  and Optimizing base model
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [6]:
# To get the number of parameters that i'm going to learn using the above architicure
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         1

#### Training and Evaluating the Model
 Training deep networks is resource intensive: depending on the size of the data, even a CNN with 3-4 successive 
 
 convolutional and pooling layers tend to take  hours to train. 
 
 Using 30 epochs and 8 layers (alternating between convolutional and pooling), our model took about 40 minutes to run.

In [7]:
# fitting my base_model with training data 
history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=15, 
                              validation_data=validation_generator, 
                              validation_steps=50)

Epoch 1/15
100/100 [==============================] - 61s 613ms/step - loss: 0.5415 - acc: 0.7500 - val_loss: 0.8047 - val_acc: 0.5625
Epoch 2/15
100/100 [==============================] - 60s 595ms/step - loss: 0.3877 - acc: 0.8213 - val_loss: 0.9015 - val_acc: 0.6250
Epoch 3/15
100/100 [==============================] - 60s 596ms/step - loss: 0.3100 - acc: 0.8628 - val_loss: 1.2263 - val_acc: 0.6250
Epoch 4/15
100/100 [==============================] - 60s 597ms/step - loss: 0.2657 - acc: 0.8847 - val_loss: 0.8716 - val_acc: 0.6250
Epoch 5/15
100/100 [==============================] - 59s 588ms/step - loss: 0.2488 - acc: 0.8959 - val_loss: 1.2822 - val_acc: 0.6250
Epoch 6/15
100/100 [==============================] - 59s 590ms/step - loss: 0.2205 - acc: 0.9100 - val_loss: 0.8608 - val_acc: 0.6250
Epoch 7/15
100/100 [==============================] - 59s 590ms/step - loss: 0.1818 - acc: 0.9272 - val_loss: 0.4767 - val_acc: 0.8750
Epoch 8/15
100/100 [==============================] - 6

In [8]:
#results_train = model.evaluate(train_images, train_labels)

In [9]:
#results_test = model.evaluate(test_images, test_labels)

In [10]:
#results_train

In [11]:
#results_test

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
start = datetime.datetime.now()


In [ ]:
rain_datagen = ImageDataGenerator(rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                        target_size=(150, 150), 
                                                        batch_size=32, 
                                                        class_mode='binary')

history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=100, 
                              validation_data=validation_generator, 
                              validation_steps=50)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print('Training with data augmentation took a total of {}'.format(elapsed))

In [ ]:
# Final Evaluation
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                  target_size=(150, 150), 
                                                  batch_size=20, 
                                                  class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [ ]:

cnn = Sequential()
#Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))
#Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))
# 2nd Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu"))
# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))
# Flatten the layer
cnn.add(Flatten())
# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 512))
cnn.add(Dense(activation = 'sigmoid', units = 1))


In [ ]:
# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.summary()

In [ ]:
cnn_history = cnn.fit_generator(train_generator,
                         steps_per_epoch = 100,
                         epochs = 25,
                         validation_data = val_generator,
                         validation_steps = 550)

In [ ]:
#results_train = cnn.evaluate(train_images, train_labels)

In [ ]:
#results_test = model.evaluate(test_images, test_labels)

In [ ]:
#results_train

In [ ]:
#results_test

### Severe OverFitting

Process
1. CNN with small dataset (Acc 71%)
2. CNN with data augmentation (Acc 80+%)
3. Transfer Learning (Acc 90+%)